<a href="https://colab.research.google.com/github/AnilZen/centpy/blob/master/notebooks/Burgers_1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burgers Equation with CentPy

### Import packages

In [ ]:
# Install the centpy package
!pip install centpy

In [ ]:
# Import numpy and centpy for the solution 
from numpy import pi, sin, cos, abs, min, max
import centpy

In [ ]:
# Imports functions from matplotlib and setup for the animation
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

### Setup the Burgers equation

We solve the inviscid Burgers' equation on the domain
$$ \partial_t u + \partial_x \left( \frac{u^2}{2}\right) = 0, $$

on the domain $(x,t)\in([0,2\pi]\times[0,10])$ with initial data
$$ u(x,0) = \sin x + \frac{1}{2} \sin\left(\frac{x}{2}\right)\,. $$

and periodic boundary conditions. The solution is computed using 400 cells and CFL number 0.75.


In [ ]:
pars = centpy.Pars1d(
    x_init=0.0,
    x_final=2.0 * pi,
    t_final=10,
    dt_out=0.1,
    J=400,
    cfl=0.75,
    scheme="sd3")

In [ ]:
class Burgers1d(centpy.Equation1d):
    def initial_data(self):
        x = self.x
        return sin(x) + 0.5 * sin(0.5 * x)

    def boundary_conditions(self, u):
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]

    def flux_x(self, u):
        return 0.5 * u**2

    def spectral_radius_x(self, u):
        return abs(u)

### Solve the equation

In [ ]:
eqn = Burgers1d(pars)
soln = centpy.Solver1d(eqn)
soln.solve()

### Animate the solution

In [ ]:
# Animation 

j0 = slice(2, -2)

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(soln.x_init,soln.x_final), ylim=(min(soln.u_n), max(soln.u_n)*1.1))

line_u, = ax.plot([], [], linewidth=1, marker='o', markersize=2)
ax.set_xlabel('x')
ax.set_ylabel('u')
plt.legend(["burgers"], loc=3, frameon=False)
 
# initialization function: plot the background of each frame
def init():
    line_u.set_data([], [])

# animation function.  This is called sequentially
def animate(i):
    line_u.set_data(soln.x[j0], soln.u_n[i,j0])
    
plt.close()
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=soln.Nt, interval=100, blit=False);
HTML(anim.to_html5_video())